In [ ]:
from local_tree_decomp import *
from help_functions import *

from local_hom_count_best import *
from local_hom_count_best_parallel import *


In [ ]:
def get_colours(n, k):
    return [i for i in range(k) for _ in range(n)]

In [ ]:
graph = graphs.CompleteBipartiteGraph(2, 4)
graph_clr = [0, 1, 1]

square = graphs.Grid2dGraph(2, 2)
square.relabel()
square_clr = [0, 1, 1, 0]

three_grid = graphs.Grid2dGraph(3, 3)
three_grid.relabel()
three_grid_clr = get_colours(3, 3)

four_grid = graphs.Grid2dGraph(4, 4)
four_grid.relabel()
four_grid_clr = get_colours(4, 4)

four_path = graphs.PathGraph(20)
target_clr = [0, 1, 2, 3]

petersen = graphs.PetersenGraph()

H = graphs.RandomGNP(100, 0.52, seed=42)

# %prun counter.count_homomorphisms_best()

# colour_counter = GraphHomomorphismCounter(square, three_grid, 2, square_clr, three_grid_clr, colourful=True)
# colourful_count = colour_counter.count_homomorphisms_best()
# print(colourful_count)

In [ ]:
counter = GraphHomomorphismCounter(graph, H)

%prun counter.count_homomorphisms_best()

In [ ]:
par_counter = ParallelGraphHomomorphismCounter(graph, H)
par_count = par_counter.count_homomorphisms_parallel()
par_count.compute()

In [ ]:
counter = GraphHomomorphismCounter(graph, H)
# count = counter.count_homomorphisms_best()
# print(count)

# %timeit -r 3 counter.count_homomorphisms_best()

In [ ]:
graph

In [ ]:
counter.nice_tree_decomp.plot()

In [ ]:
from dask.distributed import Client
client = Client()
client

In [ ]:
def best_speed_test(graph, target_graph, graph_clr, target_clr):
    # best_result = count_homomorphisms_best(graph, target_graph)
    # best_colour_result = count_homomorphisms_best_colour(graph, target_graph, graph_clr, target_clr)

    counter = GraphHomomorphismCounter(graph, target_graph)
    parallel_counter = ParallelGraphHomomorphismCounter(graph, target_graph)

    # print("Mono result:", counter.count_homomorphisms_best())
    # print("Colourful result:", colorful_counter.count_homomorphisms_best())

    print('\nSequential time:')
    %timeit -r 3 counter.count_homomorphisms_best()

    print('Parallel time:')
    par_count = parallel_counter.count_homomorphisms_parallel()
    %timeit -r 3 par_count.compute()


In [ ]:
G

In [ ]:
import random

# G = graphs.PathGraph(7)

for i in range(1, 41):
    N = i * 20
    H = graphs.RandomGNP(N, 0.52, seed=42)
    # H_color = [random.randint(0, 2) for _ in range(N)]
    print('N: ', N)
    best_speed_test(graph, H, None, None)
    print()


In [ ]:
# graph = graphs.CompleteBipartiteGraph(1, 2)
# random_graph = graphs.RandomGNP(20, 0.6, seed=42)

# counter = GraphHomomorphismCounter(graph, random_graph)
# count = counter.count_homomorphisms_best()
# print(count)

complete = graphs.CompleteGraph(9)
complete_clr = get_colours(3, 3)

counter = GraphHomomorphismCounter(three_grid, complete, 0, three_grid_clr, complete_clr, colourful=True)
print(counter.count_homomorphisms_best())

# %prun counter.count_homomorphisms_best()

In [ ]:
%load_ext line_profiler

from local_hom_count_best import count_homomorphisms_best

%lprun -f count_homomorphisms_best(graph, target_graph)

In [ ]:
g = graph

tree_decomp = g.treewidth(certificate=True)
nice_tree_decomp = make_nice_tree_decomposition(g, tree_decomp)
root = sorted(nice_tree_decomp)[0]
dir_labelled_TD = label_nice_tree_decomposition(nice_tree_decomp, root, directed=True)

In [ ]:
dir_labelled_TD

In [ ]:
node_changes(dir_labelled_TD)

In [ ]:
# colour_counter = GraphHomomorphismCounter(graph, target_graph, 2, graph_clr, target_clr, colourful=True)
# colourful_count = colour_counter.count_homomorphisms_best()
# print(colourful_count)

import random
random.seed(int(42))

graph = graphs.CompleteBipartiteGraph(1, 2)
graph_clr = [0, 1, 1]

# target_graph = graphs.CompleteGraph(3)
# target_clr = [0, 1, 2]

for i in range(1, 21):
    N = i * 5
    random_graph = graphs.RandomGNP(N, 0.5, seed=42)
    random_clr = [random.randint(0, 1) for _ in range(N)]
    print('N: ', N)
    best_speed_test(graph, random_graph, graph_clr, random_clr)
    print()

In [ ]:
import random

graph = graphs.CompleteBipartiteGraph(1, 3)
graph_clr = [0, 1, 1, 1]

target_graph = graphs.PetersenGraph()

max_num = 0

for _ in range(500):
    random_clr = [random.randint(0, 1) for _ in range(10)]
    num = count_homomorphisms_best(graph, target_graph, graph_clr, random_clr, colourful=True)
    if num > max_num:
        max_num = num

print(max_num)

In [ ]:
graph = graphs.CompleteBipartiteGraph(1, 2)
graph_clr = [0, 1, 1]

random_graph = graphs.RandomGNP(70, 0.5, seed=42)
random_clr = [random.randint(0, 1) for _ in range(N)]

colorful_counter = GraphHomomorphismCounter(graph, random_graph, 2, graph_clr, random_clr, colourful=True)
%mprun colorful_counter.count_homomorphisms_best()

In [ ]:
def blow_up_graph(original_graph, blow_up_size):
    """
    Create the blow-up of a given graph.
    
    Parameters:
    - original_graph: The original graph to be blown up.
    - blow_up_size: A dictionary mapping each vertex of the original graph to the size of the independent set replacing it.
    
    Returns:
    - A SageMath graph representing the blow-up of the original graph.
    """
    # Create an empty graph for the blow-up
    blow_up = Graph()
    
    # Step 1: Replace each vertex with an independent set
    vertex_map = {}  # Maps original vertex to the list of new vertices in the independent set
    for v in original_graph.vertices():
        new_vertices = [f'{v}_{i}' for i in range(blow_up_size[v])]
        blow_up.add_vertices(new_vertices)
        vertex_map[v] = new_vertices
    
    # Step 2: Connect the vertices between independent sets
    for u, v in original_graph.edges(labels=False):
        for new_u in vertex_map[u]:
            for new_v in vertex_map[v]:
                blow_up.add_edge(new_u, new_v)
    
    return blow_up


In [ ]:
# Define the original graph
G = graphs.PetersenGraph()

# Define the blow-up sizes for each vertex
blow_up_sizes = {v: 2 for v in G.vertices()}  # Replace each vertex with an independent set of size 2

# Create the blow-up graph
blown_up_graph = blow_up_graph(G, blow_up_sizes)

# Show the blown-up graph
blown_up_graph.show()


In [ ]:
def blow_up_graph_simple(original_graph, set_size):
    """
    Create the blow-up of a given graph with uniform independent set sizes.
    
    Parameters:
    - original_graph: The original graph to be blown up.
    - set_size: The uniform size of the independent set that replaces each vertex.
    
    Returns:
    - A SageMath graph representing the blow-up of the original graph.
    """
    blow_up = Graph()

    # Create a mapping from original vertices to new independent set vertices
    for v in original_graph.vertices():
        for i in range(set_size):
            blow_up.add_vertex(f'{v}_{i}')

    # Connect vertices in the independent sets according to the original graph's edges
    for u, v in original_graph.edges(labels=False):
        for i in range(set_size):
            for j in range(set_size):
                blow_up.add_edge(f'{u}_{i}', f'{v}_{j}')
                
    return blow_up


In [ ]:
# Define the original graph
G = graphs.CycleGraph(4)  # A simple cycle graph

# Define the uniform size for the independent sets
uniform_set_size = 3  # Replace each vertex with an independent set of size 3

# Create the blow-up graph
blown_up_graph = blow_up_graph_simple(G, uniform_set_size)

# Show the blown-up graph
blown_up_graph.show()


In [ ]:
import random

[random.randint(0, 3) for _ in range(10)]

In [ ]:
# %load_ext memory_profiler

import random
from local_hom_count_best import GraphHomomorphismCounter

N = 200
random_graph = graphs.RandomGNP(N, 0.5, seed=42)
random_clr = [random.randint(0, 1) for _ in range(N)]

colour_counter = GraphHomomorphismCounter(square, random_graph, 0, square_clr, random_clr, colourful=True)

colour_counter.count_homomorphisms_best()